In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
mid = pd.read_csv("MID ega out.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    8.790000e+02    8.790000e+02     8.790000e+02     8.790000e+02   
mean     5.157724e-11    1.652787e-12     8.772116e-13     1.906838e-12   
std      1.032898e-11    1.150466e-12     2.712876e-13     8.183822e-13   
min      2.723970e-11    6.710000e-14     3.950000e-13     7.491000e-13   
25%      4.598370e-11    9.200000e-14     6.743000e-13     1.285900e-12   
50%      4.926540e-11    2.217700e-12     7.587000e-13     1.453600e-12   
75%      5.797425e-11    2.665150e-12     1.153500e-12     2.676800e-12   
max      1.047300e-10    4.608800e-12     2.783400e-12     1.066500e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     8.790000e+02     8.790000e+02     8.790000e+02     8.790000e+02   
mean      3.391917e-12     5.696726e-12     2.973173e-13     1.832277e-12   
std       6.588591e-13     1.271360e-12     6.511995e-14     2.291729e-12   
min       2.0254

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/25 11:10:16.262,2022/11/25 11:10:16.512,2022/11/25 11:10:16.762,2022/11/25 11:10:17.012,2022/11/25 11:10:17.262,2022/11/25 11:10:17.512,2022/11/25 11:10:17.763,2022/11/25 11:10:18.263,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2022/11/25 11:10:18.584,2022/11/25 11:10:18.834,2022/11/25 11:10:19.084,2022/11/25 11:10:19.335,2022/11/25 11:10:19.585,2022/11/25 11:10:19.835,2022/11/25 11:10:20.085,2022/11/25 11:10:20.586,NaN,2.322,2.322,2.322,2.323,2.323,2.323,2.322,2.323,NaN
4,2022/11/25 11:10:20.907,2022/11/25 11:10:21.157,2022/11/25 11:10:21.407,2022/11/25 11:10:21.657,2022/11/25 11:10:21.907,2022/11/25 11:10:22.158,2022/11/25 11:10:22.408,2022/11/25 11:10:22.908,NaN,4.645,4.645,4.645,4.645,4.645,4.646,4.645,4.645,NaN
5,2022/11/25 11:10:23.229,2022/11/25 11:10:23.479,2022/11/25 11:10:23.730,2022/11/25 11:10:23.980,2022/11/25 11:10:24.230,2022/11/25 11:10:24.480,2022/11/25 11:10:24.730,2022/11/25 11:10:25.231,NaN,6.967,6.967,6.968,6.968,6.968,6.968,6.967,6.968,NaN
6,2022/11/25 11:10:25.552,2022/11/25 11:10:25.802,2022/11/25 11:10:26.052,2022/11/25 11:10:26.302,2022/11/25 11:10:26.552,2022/11/25 11:10:26.803,2022/11/25 11:10:27.053,2022/11/25 11:10:27.303,2022/11/25 11:10:27.553,9.290,9.290,9.290,9.290,9.290,9.291,9.290,9.040,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,2022/11/25 11:44:13.132,2022/11/25 11:44:13.382,2022/11/25 11:44:13.632,2022/11/25 11:44:13.882,2022/11/25 11:44:14.132,2022/11/25 11:44:14.383,2022/11/25 11:44:14.633,2022/11/25 11:44:14.883,2022/11/25 11:44:15.133,2036.870,2036.870,2036.870,2036.870,2036.870,2036.871,2036.870,2036.620,NaN
877,2022/11/25 11:44:15.454,2022/11/25 11:44:15.704,2022/11/25 11:44:15.955,2022/11/25 11:44:16.205,2022/11/25 11:44:16.455,2022/11/25 11:44:16.705,2022/11/25 11:44:16.955,2022/11/25 11:44:17.456,NaN,2039.192,2039.192,2039.193,2039.193,2039.193,2039.193,2039.192,2039.193,NaN
878,2022/11/25 11:44:17.777,2022/11/25 11:44:18.027,2022/11/25 11:44:18.277,2022/11/25 11:44:18.527,2022/11/25 11:44:18.777,2022/11/25 11:44:19.028,2022/11/25 11:44:19.278,2022/11/25 11:44:19.528,2022/11/25 11:44:19.778,2041.515,2041.515,2041.515,2041.515,2041.515,2041.516,2041.515,2041.265,NaN
879,2022/11/25 11:44:20.099,2022/11/25 11:44:20.349,2022/11/25 11:44:20.600,2022/11/25 11:44:20.850,2022/11/25 11:44:21.100,2022/11/25 11:44:21.350,2022/11/25 11:44:21.600,2022/11/25 11:44:21.850,2022/11/25 11:44:22.101,2043.837,2043.837,2043.838,2043.838,2043.838,2043.838,2043.837,2043.587,NaN


In [3]:
#reading the vaclogger file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,25/11/2022 11:10:15,7.0,-0.065449,4.010000e-09,1.120000e-09,4.996000e-09,0.000002,0.000570,0.0,-112.457,54.994,NaN,NaN
2,NaN,25/11/2022 11:10:22,14.0,-0.065228,4.000000e-09,1.120000e-09,4.996000e-09,0.000002,0.000567,0.0,-112.233,54.994,NaN,NaN
3,NaN,25/11/2022 11:10:29,21.0,-0.065622,3.970000e-09,1.110000e-09,4.996000e-09,0.000002,0.000561,0.0,-112.345,55.219,NaN,NaN
4,NaN,25/11/2022 11:10:36,28.0,-0.065795,3.940000e-09,1.100000e-09,4.996000e-09,0.000002,0.000547,0.0,-112.345,54.994,NaN,NaN
5,NaN,25/11/2022 11:10:44,36.0,-0.066289,3.930000e-09,1.100000e-09,4.996000e-09,0.000002,0.000557,0.0,-112.345,54.994,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.0,1746.0,1746.0,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1.746000e+03,1.746000e+03
mean,1101.016617,240.553356,100.086117,916.247331,913.474364,910.603746,0.0,0.0,0.0,-0.081137,0.304981,37.053883,37.068839,37.052986,-0.016647,-0.025249,-0.017080,1.045542e-07,3.049811e-07
std,603.569667,248.957862,0.014273,275.448233,279.841731,284.030718,0.0,0.0,0.0,0.131632,0.360023,11.317299,11.330990,11.340303,0.001424,0.000997,0.001016,1.139130e-07,3.600233e-07
min,1.186000,0.020000,99.900000,0.120000,0.000000,0.000000,0.0,0.0,0.0,-0.542600,0.003400,-0.082200,-0.083100,-0.076100,-0.020400,-0.026900,-0.019200,6.000000e-10,3.400000e-09
25%,579.419250,0.120000,100.080000,999.860000,999.960000,1000.000000,0.0,0.0,0.0,-0.208275,0.019100,40.137375,40.153650,40.145250,-0.017100,-0.026400,-0.018000,1.850000e-08,1.910000e-08
50%,1101.451000,0.160000,100.080000,999.900000,1000.000000,1000.060000,0.0,0.0,0.0,-0.001200,0.021000,40.867100,40.886850,40.878250,-0.016400,-0.024900,-0.017200,2.100000e-08,2.100000e-08
75%,1623.174500,499.900000,100.100000,999.920000,1000.020000,1000.060000,0.0,0.0,0.0,0.019400,0.630175,41.028550,41.049575,41.038800,-0.015600,-0.024500,-0.016500,2.091500e-07,6.301750e-07
max,2144.645000,499.900000,100.140000,999.940000,1000.040000,1001.320000,0.0,0.0,0.0,0.541200,2.775500,44.830900,46.234600,45.516800,-0.014700,-0.023200,-0.015300,5.426000e-07,2.775500e-06


### Data processing - TODO

In [5]:
#Filtering background signal - necessary only when calculating desorption yield

#Find the timestamp where emission was turned on
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:",hv.loc[hv["Comment"]=="grid repelling"])
print("Result 3:",hv.loc[hv["Comment"]=="no emission"])

x=pd.Timestamp(2022, 11, 25, 11, 13, 0)
y=pd.Timestamp(2022, 11, 25, 11, 18, 5)
z=pd.Timestamp(2022, 11, 25, 11, 8, 45)
#then find the time value closest to it in the MID scan file
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])
print("\n","Reseult 7:",time.iloc[(pd.to_datetime(time['Elapsed time, mass 2'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])
#average the signal before this timestamp and subtract it from the Em-on part
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 8:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck


#remove values under noise level from MID spectrum
mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][73:204]/h2_sens
print(p_h2.describe())
#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:73]/h2_sens
print(base_h2.describe())

Result 1:            Date          Time  Elapsed time  VMon1   VMon2  VMon3   VMon4  \
167  25/11/2022  11:12:59.714       256.306   0.08  100.08  999.6  999.56   
168  25/11/2022  11:13:00.918       257.510   0.08  100.08  999.6  999.58   
169  25/11/2022  11:13:02.111       258.703   0.08  100.08  999.6  999.58   
170  25/11/2022  11:13:03.303       259.895   0.10  100.08  999.6  999.58   

      VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
167  999.60    0.0    0.0  ...  0.9828  37.4734  37.4571  37.4626 -0.0193   
168  999.62    0.0    0.0  ...  0.9810  37.4900  37.4790  37.4842 -0.0193   
169  999.62    0.0    0.0  ...  0.9707  37.5121  37.4984  37.5014 -0.0193   
170  999.62    0.0    0.0  ...  0.9622  37.5264  37.5156  37.5170 -0.0193   

      IMon7   IMon8      Comment       hv_grid            em  
167 -0.0266 -0.0187  emission on  3.260000e-07  9.828000e-07  
168 -0.0266 -0.0187  emission on  3.332000e-07  9.810000e-07  
169 -0.0266 -0.0187  emission

count    1.310000e+02
mean     2.955335e-09
std      1.886579e-10
min      2.262212e-09
25%      2.871858e-09
50%      2.928687e-09
75%      3.003621e-09
max      3.387654e-09
Name: mass 2, corr, dtype: float64
count    7.300000e+01
mean     1.311908e-09
std      3.661868e-10
min      1.192837e-09
25%      1.206640e-09
50%      1.213302e-09
75%      1.218626e-09
max      2.758123e-09
Name: mass 2, corr, dtype: float64


C:\Users\etiirine\AppData\Local\Temp\ipykernel_20620\3248227365.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan


In [6]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Plotting

In [7]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M12')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan, EGA out on Cu @ RT, dynamic vacuum")
plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current log, EGA out on Cu @ RT, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(75, 3e-6, 'Emission on', fontdict = font)
plt.text(650, 3e-9, 'Grid repelling', fontdict = font)
plt.text(800, 1.5e-6, 'Emission on', fontdict = font)
plt.text(1050, 7e-7, '\u2190 Spike due to friction', fontdict = font)
plt.text(1530, 5.4e-8, 'Grid transparent', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution, EGA out on Cu @ RT, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.show()

# Primary desorption calculation method
### Formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [10]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = np.mean(polyfit(A,vaclog["T-CERNOX"]))
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)

#Emission current
#ignore the inrush current values
hv["em"].where(hv["em"] <= 1.2e-6, np.nan, inplace=True)
I_em = np.nanmean(hv["em"].nlargest(100))
print(I_em)

#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


1.6434268957184837e-07
304.4179192278625
0.01992516581153798
9.536929999999999e-07


In [11]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.13077189663355285 

